In [ ]:
from monster.import_data import *
from monster.features_extraction import *
from monster.scaling_and_clustering import *
from monster.scoring import *
from monster.export_output import *

%load_ext autoreload
%autoreload 2

# Create output directory

In [ ]:
## Create output director
feature_calculation_and_scaling_results, clustering_and_CLUMPs, MOnSTER_score_path, MOnSTER_analysis = create_output_directory_MOnSTER(
'MOnSTER_res_Oomycetes/')

# Data

In [ ]:
seqs_path_pos = '../data/datasets/Oomycetes_effectors_supp_table_1.1.fasta'
seqs_path_neg = '../data/datasets/Oomycetes_effectors_supp_table_1.2.fasta'
motifs_path = '../data/OOmy_ALL_motifs.txt'

## Import data
pos_dict = import_fasta_sequences_as_dict(seqs_path_pos)
neg_dict = import_fasta_sequences_as_dict(seqs_path_neg)
lst_motifs = import_list_motifs(motifs_path)
print(len(lst_motifs))

# Features calculation

In [ ]:
## Convert list of motifs to dictionary
dict_motifs = from_lst_to_dict(lst_motifs)

## The function 
df_motifs_features = feature_calculation(dict_motifs)
pos_dset_feat = feature_calculation(pos_dict)
pos_dset_feat['id'] = list(pos_dict.keys())
neg_dset_feat = feature_calculation(neg_dict)
neg_dset_feat['id'] = list(neg_dict.keys())
print(len(neg_dset_feat))

In [ ]:
# storing results into folder feature_calculation_and_scaling_results
save_df_to_directory(df_motifs_features, 'df_motifs_features', feature_calculation_and_scaling_results)
save_df_to_directory(pos_dset_feat, 'pos_dset_feat', feature_calculation_and_scaling_results)
save_df_to_directory(neg_dset_feat, 'neg_dset_feat', feature_calculation_and_scaling_results)

## Data scaling

In [ ]:
## Standard scaling (0)
## The funcion 
df_motifs_scl = features_data_scaling(df_motifs_features, 0)


In [ ]:
# storing results into folder feature_calculation_and_scaling_results
save_df_to_directory(df_motifs_scl, 'df_motifs_scl', feature_calculation_and_scaling_results)
df_motifs_scl

# Clustering
The Davies-Bouldin score try to get an evaluation of the goodness of clustering by finding a value that minimizes the distances within the points of a cluster and maximizes the distances between the clusters. For this reason choosing the first minimum of the n tries (the choosen number below) is the best treshold for dividing the clusters. Even if the global minimum is the best, in our cases the global minimum would allow too much noise to the clustering ending with tool less clusters with too many motifs in them, so choosing the first local minimum is the best choice for the scope of MOnSTER 

In [ ]:
### clustering
#
link_matrix, best_distance, dict_davies_bouldin_results, df_motifs_CLUMPs, dendogram, newick_tree = motif_clustering(
clustering_and_CLUMPs, df_motifs_scl, 6)  # the number is not the distance at which to cut, is the number of 
                                          # distances to try in order to find the best cut accordingly to D-B score
print(best_distance)
print(dict_davies_bouldin_results)






###### dict_davies_bouldin_results

In [ ]:
df_motifs_CLUMPs

In [ ]:
# storing results into folder clustering_and_CLUMPs
save_df_to_directory(df_motifs_CLUMPs, 'df_motifs_CLUMPs', clustering_and_CLUMPs) # a df for all the CLUMPs
save_lsts_motifs(df_motifs_CLUMPs, clustering_and_CLUMPs) # a df per CLUMP

In [ ]:
# if results are already stored
df_motifs_CLUMPs = pd.read_csv(f"{clustering_and_CLUMPs}/df_motifs_CLUMPs.tsv", sep=",", index_col=0)
df_motifs_CLUMPs

# Scoring

In [ ]:
## format data
df_all_motifs_all_features, pos_dset_feat, neg_dset_feat = format_input_data(df_motifs_features, df_motifs_CLUMPs,
    pos_dset_feat, neg_dset_feat)
print(len(pos_dset_feat))

In [ ]:
## find occurrences of NON REDUNDANT motifs
## ORIGINAL_scoring.py
df_general, df_start_end_position_pos, df_start_end_position_neg = find_occurrences_of_mots_in_datasets(
    df_motifs_CLUMPs, pos_dict, neg_dict)
lst_motifs_mask, motifs_counts, df_cnt_seq_per_cluster = find_occ_and_nb_seqs(df_motifs_CLUMPs, df_general)

print(len(np.unique(list(df_start_end_position_pos.seq_id))))
print(len(lst_motifs_mask))

In [ ]:
df_start_end_position_pos[["CLUMP", "seq_id"]].value_counts()
print(len(np.unique(list(df_start_end_position_pos["seq_id"]))))
save_df_to_directory(df_start_end_position_pos[["CLUMP", "seq_id"]].value_counts(), 'df_CLUMPs_in_eff', clustering_and_CLUMPs)
save_df_to_directory(df_start_end_position_neg[["CLUMP", "seq_id"]].value_counts(), "df_CLUMPs_in_non_eff", clustering_and_CLUMPs)

In [ ]:
# storing results into folder MOnSTER_score
save_df_to_directory(df_start_end_position_pos, 'df_start_end_position_pos', MOnSTER_analysis)
save_df_to_directory(df_start_end_position_neg, 'df_start_end_position_neg', MOnSTER_analysis)
df_start_end_position_neg

In [ ]:
# storing results into folder MOnSTER_score
MOnSTER_score_results, df_sign_features_p_values = MOnSTER_score(
    pos_dset_feat, neg_dset_feat, motifs_counts, df_cnt_seq_per_cluster,
    df_motifs_CLUMPs, df_all_motifs_all_features)
MOnSTER_score_results

In [ ]:
df_sign_features_p_values

In [ ]:
# storing results into folder MOnSTER_score
save_df_to_directory(df_sign_features_p_values, 'df_sign_features_p_values', MOnSTER_score_path)

In [ ]:
# summary of results
df_summary_results_monster = final_results_table(MOnSTER_score_results, df_motifs_CLUMPs, df_cnt_seq_per_cluster)
save_df_to_directory(df_summary_results_monster, 'df_summary_results_monster', MOnSTER_score_path)

In [ ]:
df_summary_results_monster